In [1]:
import os
import numpy as np
import scipy.io
from keras.models import load_model
import tqdm

## Clean

In [3]:
import os
import numpy as np
import scipy.io
import tqdm

# Set the path to the test_noisy folder
folder_path = 'data/unseenfordenoise_noaug/test_clean_stft_red-15db_Hristo'

# Initialize empty lists to store original and shift data
all_original_data_lists = {}  # Dictionary to store original data for each name
name_list = ['AS', 'MS', 'MVP', 'N', 'MR']

sampleno_range = range(161, 201)   #range(161, 201) range(1, 161)
# formatted_range = [f"{num:03}" for num in sampleno_range]

# Iterate through each name
for name in tqdm.tqdm(name_list):
    all_original_data_lists[name] = []  # List to store original data for each name

    # Iterate through each sampleno in the range
    for sampleno in sampleno_range:
        original_data_list = []

        # Iterate through the files in the test_noisy folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
            data = scipy.io.loadmat(file_path)['Segment_clean']

            if filename.startswith(f'{name}_New_{name}_{sampleno}'):
                # This is an original file
                original_data_list.append(data)

        all_original_data_lists[name].append(original_data_list)
    


# Iterate through each name and each sampleno to save the corresponding denoised spectrograms
for name in tqdm.tqdm(name_list):
    for idx, sampleno in enumerate(sampleno_range, start=1):
        denoised_spectrograms_original = []

        # Iterate through each individual .mat file for original data
        for data in all_original_data_lists[name][idx - 1]:
            # Extract real and imaginary parts
            noisy_data_real = np.real(data)
            noisy_data_imag = np.imag(data)

            # Expand dimensions
            noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

            # Reshape to (128, 64)
            denoised_data_real = noisy_data_real.reshape((128, 64))

            # Append the denoised data to the list
            denoised_spectrograms_original.append(denoised_data_real)
           

        # Concatenate all original data along the desired axis (axis=1 in this example)
        combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
        
       
        output_folder = 'data_classi/data_noaug/red-15/clean'

        # Assuming 'name' and 'sampleno' are variables you want to include in the filename
        output_filename_original = f'New_{name}_{sampleno}.mat'

        # Save the combined original data to a .mat file in the clean_spectrograms folder
        scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})


100%|██████████| 5/5 [00:00<00:00, 22.32it/s]


## Noisy

In [4]:
import os
import numpy as np
import scipy.io
import tqdm

# Set the path to the test_noisy folder
folder_path = 'data/unseenfordenoise_noaug/test_noisy_stft_red-15db_Hristo'

# Initialize empty lists to store original and shift data
all_original_data_lists = {}  # Dictionary to store original data for each name
name_list = ['AS', 'MS', 'MVP', 'N', 'MR']
sampleno_range = range(161, 201)

# Iterate through each name
for name in tqdm.tqdm(name_list):
    all_original_data_lists[name] = []  # List to store original data for each name

    # Iterate through each sampleno in the range
    for sampleno in sampleno_range:
        original_data_list = []

        # Iterate through the files in the test_noisy folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
            data = scipy.io.loadmat(file_path)['Segment_noisy']

            if filename.startswith(f'{name}_New_{name}_{sampleno}'):
                # This is an original file
                original_data_list.append(data)

        all_original_data_lists[name].append(original_data_list)


# Iterate through each name and each sampleno to save the corresponding denoised spectrograms
for name in tqdm.tqdm(name_list):
    for idx, sampleno in enumerate(sampleno_range, start=1):
        denoised_spectrograms_original = []

        # Iterate through each individual .mat file for original data
        for data in all_original_data_lists[name][idx - 1]:
            # Extract real and imaginary parts
            noisy_data_real = np.real(data)
            noisy_data_imag = np.imag(data)

            # Expand dimensions
            noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

            # Reshape to (128, 64)
            denoised_data_real = noisy_data_real.reshape((128, 64))

            # Append the denoised data to the list
            denoised_spectrograms_original.append(denoised_data_real)

        # Concatenate all original data along the desired axis (axis=1 in this example)
        combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
        output_folder = 'data_classi/data_noaug/red-15/noisy_Hristo'

        # Assuming 'name' and 'sampleno' are variables you want to include in the filename
        output_filename_original = f'New_{name}_{sampleno}.mat'

        # Save the combined original data to a .mat file in the clean_spectrograms folder
        scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})


100%|██████████| 5/5 [00:00<00:00, 21.28it/s]


## Denoise

In [6]:
from keras.layers import Lambda
import keras.backend as K
import os
import numpy as np
import scipy.io
from keras.models import load_model
import tqdm

In [8]:
#Load the model from the file
# model = load_model("model/imp_model/attn/all_pink_attnunetresi.h5")
model = load_model("model/imp_model/attn/all_red_attunet_Hristo.h5")

In [9]:
import os
import numpy as np
import scipy.io
import tqdm

# Set the path to the test_noisy folder
folder_path = 'data/unseenfordenoise_noaug/test_noisy_stft_red-5db_Hristo'

# Initialize empty lists to store original and shift data
all_original_data_lists = {}  # Dictionary to store original data for each name
name_list = ['AS', 'MS', 'MVP', 'N', 'MR']
sampleno_range = range(161, 201)

# Iterate through each name
for name in tqdm.tqdm(name_list):
    all_original_data_lists[name] = []  # List to store original data for each name

    # Iterate through each sampleno in the range
    for sampleno in sampleno_range:
        original_data_list = []

        # Iterate through the files in the test_noisy folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
            data = scipy.io.loadmat(file_path)['Segment_noisy']
            
#             if fileame.startswith(f'{name}_New_{name}_{sampleno}'): for red pink
            if filename.startswith(f'{name}_New_{name}_{sampleno}'):
                # This is an original file
                original_data_list.append(data)

        all_original_data_lists[name].append(original_data_list)


# Iterate through each name and each sampleno to save the corresponding denoised spectrograms
for name in tqdm.tqdm(name_list):
    for idx, sampleno in enumerate(sampleno_range, start=1):
        denoised_spectrograms_original = []

        # Iterate through each individual .mat file for original data
        for data in all_original_data_lists[name][idx - 1]:
            # Extract real and imaginary parts
            noisy_data_real = np.real(data)
            noisy_data_imag = np.imag(data)

            # Expand dimensions
            noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

            #predict
            denoised_data_real = model.predict(noisy_data_real)
            
            # Reshape to (128, 64)
            denoised_data_real = denoised_data_real.reshape((128, 64))

            # Append the denoised data to the list
            denoised_spectrograms_original.append(denoised_data_real)

        # Concatenate all original data along the desired axis (axis=1 in this example)
        combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
        output_folder = 'data_classi/data_noaug/red-5/denoise_unet_all_attn'
        
        os.makedirs(output_folder) if not os.path.exists(output_folder) else None

        # Assuming 'name' and 'sampleno' are variables you want to include in the filename
        output_filename_original = f'New_{name}_{sampleno}.mat'

        # Save the combined original data to a .mat file in the clean_spectrograms folder
        scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})


100%|██████████| 5/5 [01:49<00:00, 21.82s/it]


In [ ]:
1

# No Used

## Clean

In [6]:
# # Set the path to the test_noisy folder
# folder_path = 'data/unseenfordenoise_noaug/test_clean_stft_wgn0db'

# # Initialize empty lists to store original and shift data
# all_original_data_lists = []  # List to store original data for each sampleno
# all_shift_data_lists = []  # List to store all shift data for each sampleno

# name = 'MR' # 'AS' 'MS' 'MVP' 'N' 'MR'
# sampleno_range  = range(161, 201) #=[161, 171, 181, 195]
# shift_no = 4  # 4

# # Iterate through each sampleno in the range
# for sampleno in tqdm.tqdm(sampleno_range):
#     original_data_list = []
#     all_shift_data_list = []  # List to store all shift data for the current sampleno

#     # Iterate through the files in the test_noisy folder
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)

#         # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
#         data = scipy.io.loadmat(file_path)['Segment_clean']

#         if filename.startswith(f'New_{name}_{sampleno}'):
#             # This is an original file
#             original_data_list.append(data)

# #         for shift in range(1, shift_no + 1):
# #             if filename.startswith(f'New_{name}_{sampleno}_shift_{shift}_'):
# #                 # This is a shift file
# #                 # Initialize shift_data_list for the current shift
# #                 shift_data_list = all_shift_data_list[shift - 1] if len(all_shift_data_list) >= shift else []
# #                 shift_data_list.append(data)
# #                 all_shift_data_list.extend([[]] * (shift - len(all_shift_data_list)))
# #                 all_shift_data_list[shift - 1] = shift_data_list

#     all_original_data_lists.append(original_data_list)
# #     all_shift_data_lists.append([np.array(shift_data) for shift_data in all_shift_data_list])

# # Iterate through each sampleno and save the corresponding denoised spectrograms
# for idx, sampleno in tqdm.tqdm(enumerate(sampleno_range, start=1)):
#     denoised_spectrograms_original = []

#     # Iterate through each individual .mat file for original data
#     for data in all_original_data_lists[idx - 1]:
#         # Extract real and imaginary parts
#         noisy_data_real = np.real(data)
#         noisy_data_imag = np.imag(data)

#         # Expand dimensions
#         noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

#         # Reshape to (128, 64)
#         denoised_data_real = noisy_data_real.reshape((128, 64))

#         # Append the denoised data to the list
#         denoised_spectrograms_original.append(denoised_data_real)

#     # Concatenate all original data along the desired axis (axis=1 in this example)
#     combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
#     output_folder = 'data_classi/data_noaug/wgn0/clean'

#     # Assuming 'name' and 'sampleno' are variables you want to include in the filename
#     output_filename_original = f'New_{name}_{sampleno}.mat'

#     # Save the combined original data to a .mat file in the clean_spectrograms folder
#     scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})
    
# #         # Reset denoised_spectrograms for shifted data
# #     all_denoised_spectrograms_shifted = []

# #     # Iterate through each individual .mat file for shifted data
# #     for shift_data_list in all_shift_data_lists[idx - 1]:
# #         denoised_spectrograms_shifted = []

# #         for data in shift_data_list:
# #             # Extract real and imaginary parts
# #             noisy_data_real = np.real(data)
# #             noisy_data_imag = np.imag(data)

# #             # Expand dimensions
# #             noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

# #             # Reshape to (128, 64)
# #             denoised_data_real = noisy_data_real.reshape((128, 64))

# #             # Append the denoised data to the list
# #             denoised_spectrograms_shifted.append(denoised_data_real)

# #         # Concatenate all shifted data along the desired axis (axis=1 in this example)
# #         combined_shifted_data = np.concatenate(denoised_spectrograms_shifted, axis=1)
# #         all_denoised_spectrograms_shifted.append(combined_shifted_data)

# #     # Assuming 'name', 'sampleno', and 'shift_no' are variables you want to include in the filename
# #     for shift, combined_shifted_data in enumerate(all_denoised_spectrograms_shifted, start=1):
# #         output_filename_shifted = f'augmented_New_{name}_{sampleno}_shift_{shift}.mat'

# #         # Save the combined shifted data to a .mat file in the clean_spectrograms folder
# #         scipy.io.savemat(os.path.join(output_folder, output_filename_shifted), {'denoise_spe': combined_shifted_data})


100%|██████████| 40/40 [00:42<00:00,  1.06s/it]
40it [00:00, 605.77it/s]


## Noisy

In [12]:
# # Set the path to the test_noisy folder
# folder_path = 'data/unseenfordenoise_noaug/test_noisy_stft_wgn0db'

# # Initialize empty lists to store original and shift data
# all_original_data_lists = []  # List to store original data for each sampleno
# all_shift_data_lists = []  # List to store all shift data for each sampleno

# name = 'MR' # 'AS' 'MS' 'MVP' 'N' 'MR'
# sampleno_range = [161, 171, 181, 195] #range(161, 201)
# shift_no = 4  # 4

# # Iterate through each sampleno in the range
# for sampleno in tqdm.tqdm(sampleno_range):
#     original_data_list = []
#     all_shift_data_list = []  # List to store all shift data for the current sampleno

#     # Iterate through the files in the test_noisy folder
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)

#         # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
#         data = scipy.io.loadmat(file_path)['Segment_noisy']

#         if filename.startswith(f'augmented_New_{name}_{sampleno}_original_'):
#             # This is an original file
#             original_data_list.append(data)

#         for shift in range(1, shift_no + 1):
#             if filename.startswith(f'augmented_New_{name}_{sampleno}_shift_{shift}_'):
#                 # This is a shift file
#                 # Initialize shift_data_list for the current shift
#                 shift_data_list = all_shift_data_list[shift - 1] if len(all_shift_data_list) >= shift else []
#                 shift_data_list.append(data)
#                 all_shift_data_list.extend([[]] * (shift - len(all_shift_data_list)))
#                 all_shift_data_list[shift - 1] = shift_data_list

#     all_original_data_lists.append(original_data_list)
# #     all_shift_data_lists.append([np.array(shift_data) for shift_data in all_shift_data_list])

# # Iterate through each sampleno and save the corresponding denoised spectrograms
# for idx, sampleno in tqdm.tqdm(enumerate(sampleno_range, start=1)):
#     denoised_spectrograms_original = []

#     # Iterate through each individual .mat file for original data
#     for data in all_original_data_lists[idx - 1]:
#         # Extract real and imaginary parts
#         noisy_data_real = np.real(data)
#         noisy_data_imag = np.imag(data)

#         # Expand dimensions
#         noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

#         # Reshape to (128, 64)
#         denoised_data_real = noisy_data_real.reshape((128, 64))

#         # Append the denoised data to the list
#         denoised_spectrograms_original.append(denoised_data_real)

#     # Concatenate all original data along the desired axis (axis=1 in this example)
#     combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
#     output_folder = 'data_classi/data_noaug/wgn0/noisy'

#     # Assuming 'name' and 'sampleno' are variables you want to include in the filename
#     output_filename_original = f'augmented_New_{name}_{sampleno}_original.mat'

#     # Save the combined original data to a .mat file in the clean_spectrograms folder
#     scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})
    
# #         # Reset denoised_spectrograms for shifted data
# #     all_denoised_spectrograms_shifted = []

# #     # Iterate through each individual .mat file for shifted data
# #     for shift_data_list in all_shift_data_lists[idx - 1]:
# #         denoised_spectrograms_shifted = []

# #         for data in shift_data_list:
# #             # Extract real and imaginary parts
# #             noisy_data_real = np.real(data)
# #             noisy_data_imag = np.imag(data)

# #             # Expand dimensions
# #             noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

# #             # Reshape to (128, 64)
# #             denoised_data_real = noisy_data_real.reshape((128, 64))

# #             # Append the denoised data to the list
# #             denoised_spectrograms_shifted.append(denoised_data_real)

# #         # Concatenate all shifted data along the desired axis (axis=1 in this example)
# #         combined_shifted_data = np.concatenate(denoised_spectrograms_shifted, axis=1)
# #         all_denoised_spectrograms_shifted.append(combined_shifted_data)

# #     # Assuming 'name', 'sampleno', and 'shift_no' are variables you want to include in the filename
# #     for shift, combined_shifted_data in enumerate(all_denoised_spectrograms_shifted, start=1):
# #         output_filename_shifted = f'augmented_New_{name}_{sampleno}_shift_{shift}.mat'

# #         # Save the combined shifted data to a .mat file in the clean_spectrograms folder
# #         scipy.io.savemat(os.path.join(output_folder, output_filename_shifted), {'denoise_spe': combined_shifted_data})


100%|██████████| 4/4 [00:21<00:00,  5.41s/it]
4it [00:00, 574.03it/s]


# Denoised

In [19]:
# # Set the path to the test_noisy folder
# folder_path = 'data/unseenfordenoise_noaug/test_noisy_stft_wgn0db'

# # Initialize empty lists to store original and shift data
# all_original_data_lists = []  # List to store original data for each sampleno
# all_shift_data_lists = []  # List to store all shift data for each sampleno

# name = 'MR' # 'AS' 'MS' 'MVP' 'N' 'MR'
# sampleno_range =  range(161, 201) #[161, 171, 181, 195] 
# shift_no = 4  # 4

# # Iterate through each sampleno in the range
# for sampleno in tqdm.tqdm(sampleno_range):
#     original_data_list = []
#     all_shift_data_list = []  # List to store all shift data for the current sampleno

#     # Iterate through the files in the test_noisy folder
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)

#         # Assuming your data is stored in the 'Segment_noisy' field of the .mat file
#         data = scipy.io.loadmat(file_path)['Segment_noisy']

#         if filename.startswith(f'New_{name}_{sampleno}'):
#             # This is an original file
#             original_data_list.append(data)

# #         for shift in range(1, shift_no + 1):
# #             if filename.startswith(f'augmented_New_{name}_{sampleno}_shift_{shift}_'):
# #                 # This is a shift file
# #                 # Initialize shift_data_list for the current shift
# #                 shift_data_list = all_shift_data_list[shift - 1] if len(all_shift_data_list) >= shift else []
# #                 shift_data_list.append(data)
# #                 all_shift_data_list.extend([[]] * (shift - len(all_shift_data_list)))
# #                 all_shift_data_list[shift - 1] = shift_data_list

#     all_original_data_lists.append(original_data_list)
# #     all_shift_data_lists.append([np.array(shift_data) for shift_data in all_shift_data_list])

# # Iterate through each sampleno and save the corresponding denoised spectrograms
# for idx, sampleno in tqdm.tqdm(enumerate(sampleno_range, start=1)):
#     denoised_spectrograms_original = []

#     # Iterate through each individual .mat file for original data
#     for data in all_original_data_lists[idx - 1]:
#         # Extract real and imaginary parts
#         noisy_data_real = np.real(data)
#         noisy_data_imag = np.imag(data)

#         # Expand dimensions
#         noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)
    
#         denoised_data_real = model.predict(noisy_data_real)
        
#         denoised_data_real = denoised_data_real.reshape((128, 64))

#         # Append the denoised data to the list
#         denoised_spectrograms_original.append(denoised_data_real)

#     # Concatenate all original data along the desired axis (axis=1 in this example)
#     combined_original_data = np.concatenate(denoised_spectrograms_original, axis=1)
#     output_folder = 'data_classi/data_noaug/wgn0/denoise'

#     # Assuming 'name' and 'sampleno' are variables you want to include in the filename
#     output_filename_original = f'New_{name}_{sampleno}.mat'

#     # Save the combined original data to a .mat file in the clean_spectrograms folder
#     scipy.io.savemat(os.path.join(output_folder, output_filename_original), {'denoise_spe': combined_original_data})
    
# #         #Reset denoised_spectrograms for shifted data
# #     all_denoised_spectrograms_shifted = []

# #     # Iterate through each individual .mat file for shifted data
# #     for shift_data_list in all_shift_data_lists[idx - 1]:
# #         denoised_spectrograms_shifted = []

# #         for data in shift_data_list:
# #             # Extract real and imaginary parts
# #             noisy_data_real = np.real(data)
# #             noisy_data_imag = np.imag(data)

# #             # Expand dimensions
# #             noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

# #             denoised_data_real = model.predict(noisy_data_real)
            
# #             # Reshape to (128, 64)
# #             denoised_data_real = denoised_data_real.reshape((128, 64))

# #             # Append the denoised data to the list
# #             denoised_spectrograms_shifted.append(denoised_data_real)

# #         # Concatenate all shifted data along the desired axis (axis=1 in this example)
# #         combined_shifted_data = np.concatenate(denoised_spectrograms_shifted, axis=1)
# #         all_denoised_spectrograms_shifted.append(combined_shifted_data)

# #     # Assuming 'name', 'sampleno', and 'shift_no' are variables you want to include in the filename
# #     for shift, combined_shifted_data in enumerate(all_denoised_spectrograms_shifted, start=1):
# #         output_filename_shifted = f'augmented_New_{name}_{sampleno}_shift_{shift}.mat'

# #         # Save the combined shifted data to a .mat file in the clean_spectrograms folder
# #         scipy.io.savemat(os.path.join(output_folder, output_filename_shifted), {'denoise_spe': combined_shifted_data})


100%|██████████| 40/40 [00:43<00:00,  1.09s/it]
40it [00:16,  2.45it/s]


In [ ]:
# import os
# import numpy as np
# import scipy.io
# from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
# from torch import tensor

# def calculate_metrics(name, sampleno_range, clean_folder_path, denoise_folder_path):
#     results = []
#     snr_values = []
#     si_sdr_values = []
    
#     for n in name:
#         for sampleno in sampleno_range:
#             filename = f'New_{n}_{sampleno}'

#             clean_file_path = os.path.join(clean_folder_path, f'{filename}.mat')
#             denoise_file_path = os.path.join(denoise_folder_path, f'{filename}.mat')
            
#             if os.path.exists(clean_file_path) and os.path.exists(denoise_file_path):
#                 clean = scipy.io.loadmat(clean_file_path)['denoise_spe']
#                 denoise = scipy.io.loadmat(denoise_file_path)['denoise_spe']
                
#                 clean =  2 * (clean - np.min(clean)) / (np.max(clean) -np. min(clean)) - 1;
#                 denoise =  2 * (denoise - np.min(denoise)) / (np.max(denoise) -np. min(denoise)) - 1;

#                 snr = 10 * np.log10(np.sum(clean**2) / np.sum((denoise - clean)**2))
#                 snr_values.append(snr)

# #                 si_sdr = ScaleInvariantSignalDistortionRatio()
# #                 target = tensor([clean])
# #                 preds = tensor([denoise])
# #                 si_sdr_value = si_sdr(preds, target)
# #                 si_sdr_values.append(si_sdr_value.item())
                
#                 result = {'Name': n, 'SampleNo': sampleno, 'SNR': np.round(snr, 2)}
#                 results.append(result)

#     mean_snr = np.mean(snr_values)
# #     mean_si_sdr = np.mean(si_sdr_values)
    
#     print(f"Mean SNR: {mean_snr:.2f}")
# #     print(f"Mean SI-SDR: {mean_si_sdr:.2f}")

#     return results

# # Example usage
# name = ['AS', 'MS', 'MVP', 'N', 'MR']
# sampleno_range = range(161, 201)
# clean_folder_path = 'data_classi/data_noaug/wgn-5/clean'
# denoise_folder_path = 'data_classi/data_noaug/wgn-5/deoise_attnresi'

# metrics_results = calculate_metrics(name, sampleno_range, clean_folder_path, denoise_folder_path)

# for result in metrics_results:
#     print(result)
